# ChatPI

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
# This should install all the required dependencies
!pip install git+https://github.com/Manan-dev/ChatPI.git@main

# We clone the repo to load some of the example data such as contexts, etc.
!git clone https://github.com/Manan-dev/ChatPI.git


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/Manan-dev/ChatPI.git (to revision main) to /tmp/pip-req-build-gn05gogw
  Running command git clone --filter=blob:none --quiet https://github.com/Manan-dev/ChatPI.git /tmp/pip-req-build-gn05gogw
  Resolved https://github.com/Manan-dev/ChatPI.git to commit c1590e2d70a678a27f58d84ecc6be164bba8c4a1
  Preparing metadata (setup.py) ... done
fatal: destination path 'ChatPI' already exists and is not an empty directory.


In [3]:
from chatpi.utils import read_context, read_quiz
from chatpi.question_answering import run_qa
from chatpi.translation import run_tr
from chatpi.summarization import run_sum

import sys
import os


/mnt/Documents/GitHub/UTK-Classes/CS524-Natural-Language-Processing/projects/ChatPI/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## ChatBot Implementation
- Using the functions from our repository, we will implement a chatbot that will be able to answer questions about the 5 different text excerpts, translate these questions to French and back to English, and give summarizations about the text excerpts.

In [4]:
"""
qa_models = [
    # DistilBERT
    "distilbert-base-cased-distilled-squad",
    "distilbert-base-uncased-distilled-squad",
    # RoBERTa
    "deepset/roberta-base-squad2",
    "deepset/roberta-large-squad2",
    # Deberta
    "deepset/deberta-v3-base-squad2",
    "deepset/deberta-v3-large-squad2",
    # Electra
    "deepset/electra-base-squad2",
]
"""
"""
tr_models = [
    "Helsinki-NLP/opus-mt-en-fr",
    "Helsinki-NLP/opus-mt-fr-en",
    "facebook/m2m100_418M",
]
"""
"""
sum_models = [
    "sshleifer/distilbart-cnn-12-6",
    "slauw87/bart_summarisation",
    "pszemraj/pegasus-x-large-book-summary",
]
"""

# These were used for demo because they were fast. Other models, some more performant, are commented out above.
default_qa_model = "deepset/deberta-v3-base-squad2"
default_tr_model = "facebook/m2m100_418M"
default_sum_model = "sshleifer/distilbart-cnn-12-6"


def get_context_and_question(ctx_name):
    question_list = []

    for ctx_idx, (ctx_fname, ctx_text) in enumerate(
        read_context(ctx_name, basepath="./ChatPI/sections")
    ):
        ctx_fname = os.path.basename(ctx_fname)

        for q_idx, (q_text, q_answer_true) in enumerate(
            read_quiz(ctx_name, basepath="./ChatPI/sections")
        ):
            question_list.append(q_text)

        return ctx_fname, ctx_text, question_list


def run_pipelines_for_context(ctx_name):
    print(f"Loading {ctx_name} context...\n")
    _, text, questions = get_context_and_question(ctx_name)
    questions.append("Can you summarize the context?")

    print(f"{ctx_name} CONTEXT")
    print("===" * 80)
    print(text)
    print("===" * 80)
    print(f"Here is a list of questions you can ask about the {ctx_name}")

    for idx, question in enumerate(questions):
        print(f"  [{idx}] {question}")

    while True:
        print(
            "\nPlease enter a question number or type 'return' to go back to context selection\n"
        )
        user_input = input("> ")
        user_input = user_input.strip()

        if user_input == "return":
            print("Returning to context selection...\n")
            break

        elif int(user_input) == len(questions) - 1:
            print("Calling Summarization Pipeline...")
            summarized_text = run_sum(text, model=default_sum_model)
            print("> ", questions[int(user_input)])
            print("> Answer from Summarization Pipeline: ", summarized_text)

        elif int(user_input) < len(questions):
            print("Calling QA and Translation Pipelines...")
            qa_answer = run_qa(questions[int(user_input)], text, model=default_qa_model)
            french_translation_answer = run_tr(
                qa_answer, model=default_tr_model, pipeline_name="translation_en_to_fr"
            )
            english_translation_answer = run_tr(
                french_translation_answer,
                model=default_tr_model,
                pipeline_name="translation_fr_to_en",
            )
            print("> ", questions[int(user_input)])
            print("> Answer from QA Pipeline: ", qa_answer)
            print(
                "> Answer from Translation Pipeline (French): ",
                french_translation_answer,
            )
            print(
                "> Answer from Translation Pipeline (English): ",
                english_translation_answer,
            )

        else:
            print("Not a valid command or question..")

    return


In [5]:
logo = """
==================================================================
  ____ _           _   ____ ___ 
 / ___| |__   __ _| |_|  _ \_ _|
| |   | '_ \ / _` | __| |_) | | 
| |___| | | | (_| | |_|  __/| | 
 \____|_| |_|\__,_|\__|_|  |___|
 
 by. Andrei Cozma, Manan Patel, Tulsi Tailor, Zac Perry
==================================================================
    """
print(logo)
print("Welcome! Please select a context!")
print("[1] Protagonist\n[2] Antagonist\n[3] Crime\n[4] Evidence\n[5] Resolution\n")
print("Type 'help' for more commands!")

context_dictionary = {
    1: "protagonist",
    2: "antagonist",
    3: "crime",
    4: "evidence",
    5: "resolution",
}

while True:
    user_input = input("> ")

    match user_input.lower().strip():
        case "protagonist" | "1":
            run_pipelines_for_context("protagonist")

        case "antagonist" | "2":
            run_pipelines_for_context("antagonist")

        case "crime" | "3":
            run_pipelines_for_context("crime")

        case "evidence" | "4":
            run_pipelines_for_context("evidence")

        case "resolution" | "5":
            run_pipelines_for_context("resolution")

        case "help":
            print(
                "Select a context: \n[1] Protagonist\n[2] Antagonist\n[3] Crime\n[4] Evidence\n[5] Resolution\n\nOr type\nHelp -- list commands\nQuit -- exit the program\n"
            )

        case "quit":
            print("Exiting program...\n")
            break

        case _:
            print("Please enter a command\n")

print("Goodbye!")



  ____ _           _   ____ ___ 
 / ___| |__   __ _| |_|  _ \_ _|
| |   | '_ \ / _` | __| |_) | | 
| |___| | | | (_| | |_|  __/| | 
 \____|_| |_|\__,_|\__|_|  |___|
 
 by. Andrei Cozma, Manan Patel, Tulsi Tailor, Zac Perry
    
Welcome! Please select a context!
[1] Protagonist
[2] Antagonist
[3] Crime
[4] Evidence
[5] Resolution

Type 'help' for more commands!
Please enter a command

Please enter a command

Please enter a command

Please enter a command

Please enter a command

Please enter a command

Please enter a command

Please enter a command

Please enter a command

Please enter a command

Please enter a command

Please enter a command

